In [1]:
!pip install requests python-dotenv

  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)


In [1]:
import os
import base64
import requests
import re
import time
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv('OPENAI_API_KEY')

# Function to encode the image to base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Function to check if the response contains the expected answer format
def is_valid_answer(response_text):
    return re.search(r"resposta=\['([A-E])'\]", response_text) is not None

# Function to read correct answers from a file into a dictionary
def read_correct_answers(file_path):
    correct_answers = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            key, value = line.strip().split(': ')
            correct_answers[int(key)] = value.strip()
    return correct_answers

# Compare the answers and calculate the score
def calculate_score(answers_dict, correct_answers):
    total_questions = len(correct_answers)
    correct_count = sum(answers_dict.get(key) == correct_answers.get(key) for key in correct_answers)
    return (correct_count / total_questions) * 100

# Path to your folder containing images
folder_path = "questions"

# Create a folder for storing reasoning text files
os.makedirs("reasoning", exist_ok=True)

# List all files in the folder
files = os.listdir(folder_path)

# Filter out only PNG files
png_files = [file for file in files if file.endswith('.png')][0:2]

# Dictionary to store the answers
answers_dict = {}

# Counter for total retries
total_retries = 0

# Start time
start_time = time.time()

# Iterate over each PNG file
for file in png_files:
    # Extract the question number from the file name
    question_number_match = re.search(r"(\d+).png", file)
    if question_number_match:
        current_key = int(question_number_match.group(1))
    else:
        print(f"Warning: File '{file}' does not follow the naming convention and will be skipped.")
        continue

    image_path = os.path.join(folder_path, file)

    # Getting the base64 string
    base64_image = encode_image(image_path)

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    payload = {
        "model": "gpt-4-vision-preview",
        "messages": [
          {
            "role": "user",
            "content": [
              {
                "type": "text",
                "text": "Leia atentamente a questão contida na imagem. Resolva a questão pensando passo a passo. \
                  Toda questão possui uma única resposta correta. Ao final, na última linha, forneça a sua resposta no seguinte formato:\
                  resposta=['A']"
              },
              {
                "type": "image_url",
                "image_url": {
                  "url": f"data:image/png;base64,{base64_image}"
                }
              }
            ]
          }
        ],
        "max_tokens": 1500
    }

    retries = 0
    while True:
        response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
        full_response = response.json().get('choices', [{}])[0].get('message', {}).get('content', 'No answer found')

        # Save the full answer to a text file with UTF-8 encoding
        answer_file_path = os.path.join("reasoning", file.replace('.png', '.txt'))
        with open(answer_file_path, 'w', encoding='utf-8') as answer_file:
            answer_file.write(full_response)

        if is_valid_answer(full_response):
            # Extract the final answer from the response
            match = re.search(r"resposta=\['([A-E])'\]", full_response)
            answer = match.group(1) if match else 'Invalid format'
            answers_dict[current_key] = answer
            break
        else:
            retries += 1
            total_retries += 1
            if retries >= 3:  # Max retries per image
                answers_dict[current_key] = 'No valid answer found after 3 retries'
                break

# After processing all questions
# Read the correct answers from a file
correct_answers_file_path = "marking_scheme.txt"
correct_answers = read_correct_answers(correct_answers_file_path)

# Calculate the final score
final_score = calculate_score(answers_dict, correct_answers)

# Calculate total elapsed time
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Total Math Questions Processed: {len(png_files)}")
print(f"Total Retries: {total_retries}")
print(f"Final Score: {final_score}%")
print(f"Total Time Elapsed: {elapsed_time:.2f} seconds")


Total Math Questions Processed: 45
Total Retries: 5
Final Score: 35.55555555555556%
Total Time Elapsed: 758.12 seconds
